# SSVQE (Noiseless) Comparisons

Pure package client for comparing SSVQE performance across:

1) Optimizers (for fixed ansatz, optimizer-specific step sizes)
2) Ansatzes   (for fixed optimizer)
3) Full grid  (all ansatz × optimizer combos)
4) Pick a "best" config using an explicit excited-state target policy
5) Validate top configs across multiple seeds (mean ± std)

Notes:
- This notebook intentionally avoids manual Hamiltonian construction and direct diagonalization.
- Rankings are based on the returned SSVQE energies and simple internal-consistency diagnostics.

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Sequence, Tuple

import numpy as np

from vqe.ssvqe import run_ssvqe

## Configuration

In [ ]:
# -----------------------------
# Problem / run configuration
# -----------------------------
molecule = "H2"
num_states = 2
noisy = False

# Only used if noisy=True
depolarizing_prob = 0.0
amplitude_damping_prob = 0.0
noise_model = None

# SSVQE weights (None => default [1,2,3,...])
weights: Optional[Sequence[float]] = None

# Reference states policy:
# - None: run_ssvqe decides defaults (UCC => HF+excitations, otherwise computational basis).
# - Or pass explicit bitstrings of length n_wires (if you want full control).
reference_states = None

# Optimization budget
steps = 250
seed = 0

# -----------------------------
# Optimizer sweep configuration
# -----------------------------
optimizers = ["Adam", "GradientDescent", "Momentum", "Nesterov", "RMSProp", "Adagrad"]
stepsize_map = {
    "Adam": 0.2,
    "GradientDescent": 0.05,
    "Momentum": 0.1,
    "Nesterov": 0.1,
    "RMSProp": 0.1,
    "Adagrad": 0.2,
}

fixed_ansatz = "UCCSD"

# -----------------------------
# Ansatz sweep configuration
# -----------------------------
ansatzes = [
    "UCC-S",
    "UCC-D",
    "UCCSD",
    "Minimal",
    "RY-CZ",
    "TwoQubit-RY-CNOT",
    "StronglyEntanglingLayers",
]

fixed_optimizer = "Adam"
fixed_stepsize = stepsize_map[fixed_optimizer]

# -----------------------------
# "Best config" target policy
# -----------------------------
# This is an internal policy for ranking configurations when exact diagonalization is not used.
excited_target_mode = "first"  # "first" or "low_k"
excited_target_k = 6           # used only if mode="low_k"

## Warm-up: print basic system metadata (from the engine)

We run a very short SSVQE job to surface whatever metadata the implementation returns
(e.g., number of qubits, basis, mapping). This keeps the notebook "package-client only".

In [ ]:
_warm = run_ssvqe(
    molecule=molecule,
    num_states=num_states,
    weights=weights,
    ansatz_name=fixed_ansatz,
    optimizer_name=fixed_optimizer,
    steps=1,
    stepsize=float(fixed_stepsize),
    seed=int(seed),
    noisy=noisy,
    depolarizing_prob=depolarizing_prob,
    amplitude_damping_prob=amplitude_damping_prob,
    noise_model=noise_model,
    reference_states=reference_states,
    plot=False,
    force=False,
)

print(f"Molecule: {molecule}")
if isinstance(_warm, dict):
    if "num_qubits" in _warm:
        print(f"Qubits:   {int(_warm['num_qubits'])}")
    for k in ("basis", "mapping", "backend", "device", "n_wires"):
        if k in _warm:
            print(f"{k}: { _warm[k] }")

## Run wrapper + table printer

In [ ]:
@dataclass(frozen=True)
class SSVQERow:
    ansatz: str
    optimizer: str
    stepsize: float
    E_final: Tuple[float, ...]  # sorted, length = num_states


def _run_ssvqe_final_energies(
    *,
    ansatz_name: str,
    optimizer_name: str,
    stepsize: float,
    seed: int,
) -> Tuple[float, ...]:
    """
    Run SSVQE once and return final energies for all states as a sorted tuple.
    """
    out = run_ssvqe(
        molecule=molecule,
        num_states=num_states,
        weights=weights,
        ansatz_name=ansatz_name,
        optimizer_name=optimizer_name,
        steps=steps,
        stepsize=stepsize,
        seed=seed,
        noisy=noisy,
        depolarizing_prob=depolarizing_prob,
        amplitude_damping_prob=amplitude_damping_prob,
        noise_model=noise_model,
        reference_states=reference_states,
        plot=False,
        force=False,
    )

    trajs = out.get("energies_per_state", None) if isinstance(out, dict) else None
    if trajs is None:
        raise RuntimeError("run_ssvqe did not return 'energies_per_state' in output dict.")

    finals_raw = np.array([float(traj[-1]) for traj in trajs], dtype=float)
    finals = tuple(float(x) for x in np.sort(finals_raw))
    return finals


def _print_rows(rows: Sequence[SSVQERow], *, title: str) -> None:
    if not rows:
        print(f"\n{title}\n(no rows)\n")
        return

    headers = ["Ansatz", "Optimizer", "Stepsize"] + [f"E{i} (Ha)" for i in range(num_states)]

    def fmt_row(r: SSVQERow) -> List[str]:
        parts = [r.ansatz, r.optimizer, f"{r.stepsize:g}"]
        parts += [f"{e:+.10f}" for e in r.E_final]
        return parts

    data = [fmt_row(r) for r in rows]
    widths = [len(h) for h in headers]
    for row in data:
        widths = [max(w, len(cell)) for w, cell in zip(widths, row)]

    print("\n" + title)
    line = "  ".join(h.ljust(w) for h, w in zip(headers, widths))
    print(line)
    print("-" * len(line))
    for row in data:
        print("  ".join(cell.ljust(w) for cell, w in zip(row, widths)))

## 1) Optimizer comparison (fixed ansatz)

In [ ]:
optimizer_rows: List[SSVQERow] = []

for opt in optimizers:
    ss = float(stepsize_map[opt])
    print(f"Running: ansatz={fixed_ansatz}, optimizer={opt}, stepsize={ss} ...")
    finals = _run_ssvqe_final_energies(
        ansatz_name=fixed_ansatz,
        optimizer_name=opt,
        stepsize=ss,
        seed=int(seed),
    )
    optimizer_rows.append(SSVQERow(fixed_ansatz, opt, ss, finals))

optimizer_rows_sorted = sorted(
    optimizer_rows,
    key=lambda r: r.E_final[1] if num_states > 1 else r.E_final[0],
)
_print_rows(optimizer_rows_sorted, title=f"SSVQE Optimizer Comparison (ansatz={fixed_ansatz})")

## 2) Ansatz comparison (fixed optimizer)

In [ ]:
ansatz_rows: List[SSVQERow] = []

for ans in ansatzes:
    print(f"Running: ansatz={ans}, optimizer={fixed_optimizer}, stepsize={fixed_stepsize} ...")
    finals = _run_ssvqe_final_energies(
        ansatz_name=ans,
        optimizer_name=fixed_optimizer,
        stepsize=float(fixed_stepsize),
        seed=int(seed),
    )
    ansatz_rows.append(SSVQERow(ans, fixed_optimizer, float(fixed_stepsize), finals))

ansatz_rows_sorted = sorted(
    ansatz_rows,
    key=lambda r: r.E_final[1] if num_states > 1 else r.E_final[0],
)
_print_rows(ansatz_rows_sorted, title=f"SSVQE Ansatz Comparison (optimizer={fixed_optimizer})")

## 3) Full grid (all ansatz × optimizer combos)

In [ ]:
grid_rows: List[SSVQERow] = []

for ans in ansatzes:
    for opt in optimizers:
        ss = float(stepsize_map[opt])
        print(f"Running: ansatz={ans}, optimizer={opt}, stepsize={ss} ...")
        finals = _run_ssvqe_final_energies(
            ansatz_name=ans,
            optimizer_name=opt,
            stepsize=ss,
            seed=int(seed),
        )
        grid_rows.append(SSVQERow(ans, opt, ss, finals))

grid_rows_sorted = sorted(
    grid_rows,
    key=lambda r: (r.E_final[1] if num_states > 1 else r.E_final[0], r.E_final[0]),
)
_print_rows(grid_rows_sorted, title="SSVQE Full Grid (all ansatz × optimizer combos)")

## 4) Choose the "best" configuration (internal policy)

Without exact diagonalization, we rank configurations using:
- E0_final (lower is better)
- excited-state target depends on `excited_target_mode`:
    * "first": use E1_final directly (lower is better)
    * "low_k": use E1_final with a mild penalty if the gap is implausibly small/negative

Score:
- "first":  score = E0 + E1
- "low_k": score = E0 + E1 + gap_penalty

In [ ]:
def _gap_penalty(E0: float, E1: float) -> float:
    # Penalize state order inversions or near-degeneracy (heuristic).
    gap = float(E1 - E0)
    if gap < 0:
        return 1.0 + abs(gap)
    if gap < 1e-3:
        return 1e-2
    return 0.0


def _score_row(r: SSVQERow) -> Dict[str, Any]:
    E0 = float(r.E_final[0])
    E1 = float(r.E_final[1]) if len(r.E_final) > 1 else float("nan")

    mode = str(excited_target_mode).strip().lower()
    if mode == "first":
        score = E0 + E1
        pen = 0.0
    elif mode == "low_k":
        # In a no-exact-spectrum setting, treat "low_k" as "prefer low E1 but reject pathological gaps".
        pen = _gap_penalty(E0, E1)
        score = E0 + E1 + pen
    else:
        raise ValueError("excited_target_mode must be 'first' or 'low_k'.")

    return {
        "row": r,
        "E0": E0,
        "E1": E1,
        "gap": float(E1 - E0) if np.isfinite(E1) else float("nan"),
        "penalty": float(pen),
        "score": float(score),
    }


scored = [_score_row(r) for r in grid_rows]
scored_sorted = sorted(scored, key=lambda x: (x["score"], x["penalty"], x["gap"]))

best = scored_sorted[0]
rbest: SSVQERow = best["row"]

print(f"\nTarget policy: excited_target_mode={excited_target_mode!r}, excited_target_k={excited_target_k}")
print("\nBest configuration (by internal score):")
print(f"  Ansatz:    {rbest.ansatz}")
print(f"  Optimizer: {rbest.optimizer}")
print(f"  Stepsize:  {rbest.stepsize:g}")
print(f"  E0_final:  {best['E0']:+.10f} Ha")
print(f"  E1_final:  {best['E1']:+.10f} Ha")
print(f"  Gap:       {best['gap']:.3e} Ha")
print(f"  Penalty:   {best['penalty']:.3e}")
print(f"  Score:     {best['score']:+.10f}")

top_k = 10
print(f"\nTop {top_k} configurations:")
for i, x in enumerate(scored_sorted[:top_k], start=1):
    rr: SSVQERow = x["row"]
    print(
        f"{i:>2}. ansatz={rr.ansatz:<22} optimizer={rr.optimizer:<14} stepsize={rr.stepsize:<5g}  "
        f"E0={x['E0']:+.10f}  E1={x['E1']:+.10f}  gap={x['gap']:.2e}  pen={x['penalty']:.2e}  score={x['score']:+.10f}"
    )

## 5) Validate top configurations across multiple seeds (mean ± std)

In [14]:
seeds = np.arange(5)
top_k_validate = 5
top_configs = scored_sorted[:top_k_validate]

def _mean_std(xs: Sequence[float]) -> Tuple[float, float]:
    a = np.asarray(xs, dtype=float)
    if len(a) <= 1:
        return float(a.mean()), 0.0
    return float(a.mean()), float(a.std(ddof=1))


val_rows: List[Dict[str, Any]] = []

for i, x in enumerate(top_configs, start=1):
    rr: SSVQERow = x["row"]

    E0s: List[float] = []
    E1s: List[float] = []
    gaps: List[float] = []
    pens: List[float] = []
    scores: List[float] = []

    print(
        f"\nValidating {i}/{len(top_configs)}: ansatz={rr.ansatz}, optimizer={rr.optimizer}, "
        f"stepsize={rr.stepsize:g}, seeds={list(seeds)}"
    )

    for s in seeds:
        finals = _run_ssvqe_final_energies(
            ansatz_name=rr.ansatz,
            optimizer_name=rr.optimizer,
            stepsize=float(rr.stepsize),
            seed=int(s),
        )
        E0 = float(finals[0])
        E1 = float(finals[1]) if len(finals) > 1 else float("nan")
        gap = float(E1 - E0)
        pen = _gap_penalty(E0, E1) if str(excited_target_mode).strip().lower() == "low_k" else 0.0
        score = E0 + E1 + pen

        E0s.append(E0)
        E1s.append(E1)
        gaps.append(gap)
        pens.append(pen)
        scores.append(score)

    E0_mean, E0_std = _mean_std(E0s)
    E1_mean, E1_std = _mean_std(E1s)
    gap_mean, gap_std = _mean_std(gaps)
    pen_mean, pen_std = _mean_std(pens)
    score_mean, score_std = _mean_std(scores)

    val_rows.append(
        {
            "ansatz": rr.ansatz,
            "optimizer": rr.optimizer,
            "stepsize": rr.stepsize,
            "E0_mean": E0_mean,
            "E0_std": E0_std,
            "E1_mean": E1_mean,
            "E1_std": E1_std,
            "gap_mean": gap_mean,
            "gap_std": gap_std,
            "pen_mean": pen_mean,
            "pen_std": pen_std,
            "score_mean": score_mean,
            "score_std": score_std,
        }
    )

val_rows_sorted = sorted(val_rows, key=lambda r: (r["score_mean"], r["score_std"], r["gap_std"]))

print("\nMulti-seed validation summary (sorted by mean score):")
hdr = (
    f"{'Ansatz':<22}  {'Optimizer':<14}  {'SS':<5}  "
    f"{'E0 mean±std':<26}  {'E1 mean±std':<26}  "
    f"{'gap mean±std':<26}  {'pen mean±std':<22}  {'score mean±std':<24}"
)
print(hdr)
print("-" * len(hdr))

for r in val_rows_sorted:
    print(
        f"{r['ansatz']:<22}  {r['optimizer']:<14}  {r['stepsize']:<5g}  "
        f"{r['E0_mean']:+.10f}±{r['E0_std']:.2e}  "
        f"{r['E1_mean']:+.10f}±{r['E1_std']:.2e}  "
        f"{r['gap_mean']:.2e}±{r['gap_std']:.2e}  "
        f"{r['pen_mean']:.2e}±{r['pen_std']:.2e}  "
        f"{r['score_mean']:+.10f}±{r['score_std']:.2e}"
    )

best_mean = val_rows_sorted[0]
print("\nBest configuration by mean score across seeds:")
print(f"  Ansatz:    {best_mean['ansatz']}")
print(f"  Optimizer: {best_mean['optimizer']}")
print(f"  Stepsize:  {best_mean['stepsize']}")
print(f"  E0:        {best_mean['E0_mean']:+.10f} ± {best_mean['E0_std']:.2e}")
print(f"  E1:        {best_mean['E1_mean']:+.10f} ± {best_mean['E1_std']:.2e}")
print(f"  gap:       {best_mean['gap_mean']:.2e} ± {best_mean['gap_std']:.2e}")
print(f"  penalty:   {best_mean['pen_mean']:.2e} ± {best_mean['pen_std']:.2e}")
print(f"  score:     {best_mean['score_mean']:+.10f} ± {best_mean['score_std']:.2e}")